In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import glob
import cv2
#import seaborn as sns
%matplotlib inline

In [2]:
from opmap.RawCam import RawCam
from opmap.VmemMap import VmemMap
from opmap.PhaseMap import PhaseMap
from opmap.PhaseVarianceMap import PhaseVarianceMap
from opmap.util import makeMovie

In [3]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

# 興奮前面・興奮後面マップを拡大

In [1]:
dir_names = ['20171025-1']

for dir_name in dir_names:
    time = 1000
    width = 5
    threshold = 0.01
    Ina = np.zeros((time, 200, 200))
    hj = np.zeros((time, 200, 200))
    save_dir = '/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/20171113'
    
    if not os.path.exists(os.path.join(save_dir, 'wf_wt', dir_name)):
        os.makedirs(os.path.join(save_dir, 'wf_wt', dir_name))
    
    for i in range(time):
        num = '{0:04d}'.format(i)
        Ina[i, :, :] = np.load('/mnt/recordings/SimulationResults/2D/%s/cell_%s/%s' % (dir_name, num, 'xina.npy')).reshape((200, 200))
        a = np.load('/mnt/recordings/SimulationResults/2D/%s/cell_%s/%s' % (dir_name, num, 'h.npy')).reshape((200, 200))
        b = np.load('/mnt/recordings/SimulationResults/2D/%s/cell_%s/%s' % (dir_name, num, 'j.npy')).reshape((200, 200))
        hj[i, :, :] = a*b
    
    for i in range(time):
        if i < width+1:
            continue
        num = '{0:06d}'.format(i-width-1)
        im_out = np.zeros([200, 200, 3], dtype=np.uint8)
        ans = np.zeros((1,200, 200))
        for j in range(width):
            ans += ((hj[j-1-width+i:j-width+i, :, :] < threshold) * (hj[j-width+i:j+1-width+i, :, :] >= threshold))
        im_out[:, :, 0] = (np.sum((Ina[i-width:i, :, :] < -1), axis=0) > 0)*255
        im_out[:, :, 1] = ans[0, :, :]*255
        im_out[:, :, 2] = ans[0, :, :]*255
        plt.imshow(im_out[75:125, 100:150, :])
        plt.text(5,10, "%s (ms)" % "{0:04d}".format(i-width-1), fontsize=17, color='w')
        plt.xticks([])
        plt.yticks([])
        plt.savefig(os.path.join(save_dir, 'wf_wt/%s/%s.png' % (dir_name, num)))
        plt.close()
    
#    subprocess.call('ffmpeg -r 30 -y -i "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/20171106/threshold/{0}/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/20171106/threshold/{0}.mp4"'.format(dir_name), shell=True)

NameError: name 'np' is not defined